In [1]:
#importar librerias pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

#crear sesion spark
spark = SparkSession.builder.appName("Analisis_Opinion_PeruARusia")\
            .config("spark.some.config.option","some-value")\
            .getOrCreate()

In [2]:
#cargar dataset en DataFrame
tuits_peruArusia = spark.read.csv("data/PeruARusia2018.csv", inferSchema = True, header = True)
tuits_peruArusia.show(2)

+---------+---------------+----------------+--------------------+--------+--------------------+
|polaridad|        id_tuit|           fecha|            consulta| usuario|               texto|
+---------+---------------+----------------+--------------------+--------+--------------------+
|        0|120151009097021|2015-10-09 20:00|#PerdimosComoSiempre|@efatatv|la llorona http:/...|
|        0|120151009097025|2015-10-09 16:00|#PerdimosComoSiempre|@efatatv|Nunca podrán http...|
+---------+---------------+----------------+--------------------+--------+--------------------+
only showing top 2 rows



In [3]:
#seleccionar los datos necesarios para el modelo de aprendizaje
tuitsData = tuits_peruArusia.select(col("polaridad").cast("Int").alias("label"),"texto")
tuitsData.show(2)

+-----+--------------------+
|label|               texto|
+-----+--------------------+
|    0|la llorona http:/...|
|    0|Nunca podrán http...|
+-----+--------------------+
only showing top 2 rows



In [23]:
tuitsData.show(10, False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------+
|label|texto                                                                                                                                        |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------+
|0    |la llorona http:// ow.ly/TdYHX #perdimoscomosiempre #NuncaFaltaUnAmigoQue #Perfect #BuenViernes #MaduroTuPuebloTeApoya #Lima #Peru           |
|0    |Nunca podrán http:// ow.ly/TdZq4 #perdimoscomosiempre #NuncaFaltaUnAmigoQue #Perfect #BuenViernes #MaduroTuPuebloTeApoya #Lima #Peru         |
|1    |Jugamos como nunca y #PerdimosComoSiempre ..... Pero igual hasta las finales con ustedes....los veremos en #Rusia2018 ... Recién empezamos   |
|1    |#PerdimosComoSiempre asi es el deporte pero aun seguimos alentando pic.twitter.com/CvVWcKSQeP

In [4]:
#1.-limpiando texto, remplazando vocales atildadas por vocales simples
dataSinTilde = tuitsData.select(translate(col("texto"),"áéíóú","aeiou")\
                                .alias("textoSinTilde")\
                                ,col("label"))

#2 y 3.-limpiando texto, quitando @usuarios y #etiqueta
patron = "@[A-Za-z0-9]+|#[A-Za-z0-9]+"
dataSinUserEtiqueta = dataSinTilde.select(regexp_replace(col("textoSinTilde"),patron,"")\
                                          .alias("textoSinUsEt")\
                                          ,col("label"))

#4.-limpiando texto, quitando pic.twitter.com/... goo.gl/... 
# twitter.com/../.. /status/... fb.me/...
patron="pic.twitter.com/[A-Za-z0-9]+\
|goo.gl/[A-Za-z0-9]+\
|twitter.com/[A-Za-z0-9]+/[A-Za-z0-9]+\
|twitter.com/[A-Za-z0-9]+\
|/status/[A-Za-z0-9]+\
|fb.me/[A-Za-z0-9]+"
dataSinURL = dataSinUserEtiqueta.select(regexp_replace(col("textoSinUsEt"),patron,"")\
                                        .alias("textoSinURL")\
                                        ,col("label"))

#5.-limpiando texto, quitando tatus/... word.word.com/ http:// https://
patron="http://|https://|[A-Za-z0-9]+/[A-Za-z0-9]+\
|[A-Za-z0-9]+.[A-Za-z0-9]+.[A-Za-z0-9]+/[A-Za-z0-9]+"
dataSinURLs = dataSinURL.select(regexp_replace(col("textoSinURL"),patron,"")\
                                .alias("textoSinURLs")\
                                ,col("label"))

#6.-limpiando texto, quitando simbolos, signos de puntuacion
patron="\"|\.|,|;|:|¿|\?|¡|!|=|-|/|…"
dataSinPunt = dataSinURLs.select(regexp_replace(col("textoSinURLs"),patron,"")\
                                 .alias("textoSinPunt")\
                                 ,col("label"))

#7.-limpiando texto, quitando espacios en blanco demas y remplazarlo por un espacio
patron=" +"
dataLimpio = dataSinPunt.select(regexp_replace(col("textoSinPunt"),patron," ")\
                                .alias("text")\
                                ,col("label"))
dataLimpio.show(2, False)

+----------------+-----+
|text            |label|
+----------------+-----+
|la llorona ow   |0    |
|Nunca podran ow |0    |
+----------------+-----+
only showing top 2 rows



In [5]:
dataLimpio.show(10, False)

+-------------------------------------------------------------------------------------------------+-----+
|text                                                                                             |label|
+-------------------------------------------------------------------------------------------------+-----+
|la llorona ow                                                                                    |0    |
|Nunca podran ow                                                                                  |0    |
|Jugamos como nunca y Pero igual hasta las finales con ustedeslos veremos en Recien empezamos     |1    |
| asi es el deporte pero aun seguimos alentando                                                   |1    |
| eso es historia repetida pero nunca perderemos la FE                                            |1    |
|Jugamos con rebeldia garra inteligencia pero                                                     |1    |
|Yo ante cada partido de Peru frente a cualqui

In [6]:
#Dividir el dataset en entrenamiento=70% y prueba=30%
tuitsDividido = dataLimpio.randomSplit([0.7, 0.3])
tuitsEntrenamiento = tuitsDividido[0]
tuitsPrueba = tuitsDividido[1]
nroTuitsEntrenamiento = tuitsEntrenamiento.count()
nroTuitsPrueba = tuitsPrueba.count()
print("Tuits de Entrenamiento :",nroTuitsEntrenamiento, "Tuits de Prueba :",nroTuitsPrueba)

Tuits de Entrenamiento : 1974 Tuits de Prueba : 830


In [7]:
#Preparar Datos de Entrenamiento: Tokenizacion
tokenizer = Tokenizer(inputCol = "text", outputCol="palabras")
tokenizedData = tokenizer.transform(tuitsEntrenamiento)
tokenizedData.select("palabras","label").show(5, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|palabras                                                                                                                                                                                                                                                                                                                |label|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[, *gareca, agradeciendo, la, actitu

In [8]:
tokenizedData.select("palabras","label").show(11)

+--------------------+-----+
|            palabras|label|
+--------------------+-----+
|[, *gareca, agrad...|    0|
|[, 1982, era, jov...|    1|
|[, 2017, esa, fec...|    1|
|[, 2017, esa, fec...|    1|
|[, a, cualquier, ...|    0|
|[, atv, ya, no, v...|    0|
|[, ahora, a, espe...|    0|
|[, ahora, pues, d...|    0|
|[, algun, dia, de...|    0|
|[, aqui, no, es, ...|    0|
|[, arbitro, de, m...|    0|
+--------------------+-----+
only showing top 11 rows



In [9]:
#Preparar Datos: quitar palabras sin importancia de spanish
spanishStopWords = StopWordsRemover.loadDefaultStopWords("spanish")
swr = StopWordsRemover(stopWords = spanishStopWords, inputCol = tokenizer.getOutputCol(), outputCol="palabrasUtil")
swrTuitsData = swr.transform(tokenizedData)

In [10]:
swrTuitsData.select("palabrasUtil").show(6, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|palabrasUtil                                                                                                                                                                                                             |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[, *gareca, agradeciendo, actitud, seleccion*, metieron, gol, metiste, benavente]                                                                                                                                        |
|[, 1982, joven, aun, recuerdo, alegria, mundial, padre, volvi, verlo, hijos, solo, ideal, millones, corazones, vibrando

In [11]:
#Convertir las palabras Utiles a caracteristicas numericas
hashTF = HashingTF(inputCol=swr.getOutputCol(),outputCol="features")
numericTrainData = hashTF.transform(swrTuitsData).select("label","palabrasUtil","features")

In [12]:
numericTrainData.show(truncate=False, n=3)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|palabrasUtil                                                                                                                                                                                                             |features                                                                                                                                                                                                                                    

In [13]:
#Entrenar el modelo con los datos de entrenamiento
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)
print("El modelo esta entrenado!")

El modelo esta entrenado!


In [14]:
#Preparar datos de entrenamiento
tokenizedTest = tokenizer.transform(tuitsPrueba)
swrTuitsDataTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(swrTuitsDataTest).select("label","palabrasUtil","features")
numericTest.show(truncate = False, n = 3)

+-----+----------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|palabrasUtil                                                                                        |features                                                                                                                                                           |
+-----+----------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[, &lt&lt&lt, q, mala, defensa, aun, mejora, copa, america, colmo, perdio, peru, tremendos, errores]|(262144,[23574,48982,93899,103970,108672,122591,146326,156299,190226,2048

In [15]:
#Predecir los datos de prueba y calcular la presicion del modelo
prediction = model.transform(numericTest)
predictionFinal = prediction.select("palabrasUtil","prediction","label")
predictionFinal.show(n=5)
correctPrediction = predictionFinal.filter(predictionFinal["label"]==predictionFinal["prediction"]).count()
totalData = predictionFinal.count()
accuracy = correctPrediction/totalData
print("Predicciones correctas :",correctPrediction, "Total Datos :", totalData, "Accuracy :",accuracy)

+--------------------+----------+-----+
|        palabrasUtil|prediction|label|
+--------------------+----------+-----+
|[, &lt&lt&lt, q, ...|       0.0|    0|
|[, 6, goles, 4, e...|       0.0|    0|
|[, cualquier, lad...|       0.0|    0|
|[, qui, viene, ve...|       1.0|    1|
|[, gusta, futbol,...|       0.0|    0|
+--------------------+----------+-----+
only showing top 5 rows

Predicciones correctas : 678 Total Datos : 830 Accuracy : 0.8168674698795181


In [16]:
true_positives = predictionFinal.filter('label==1 and prediction==1.0').count()
print(true_positives)

415


In [17]:
true_negatives = predictionFinal.filter('label==0 and prediction==0.0').count()
print(true_negatives)

263


In [18]:
false_positives = predictionFinal.filter('label==0 and prediction==1.0').count()
print(false_positives)
false_negatives = predictionFinal.filter('label==1 and prediction==0.0').count()
print(false_negatives)

117
35


In [19]:
recall = float(true_positives)/(true_positives + false_negatives)
print("Recall :", recall)
precision = float(true_positives)/(true_positives + false_positives)
print("Precision :", precision)
accuracy = float(true_positives + true_negatives)/(totalData)
print("Accuracy :", accuracy)

Recall : 0.9222222222222223
Precision : 0.7800751879699248
Accuracy : 0.8168674698795181


In [22]:
F1Score = 2*float(precision*recall)/(precision + recall)
print("F1 Score :", F1Score)

F1 Score : 0.845213849287169
